## Regressão logística para classificação de imagens

Você irá construir um classificador baseado em regressão logística para reconhecer gatos. Este exercício irá mostrar os passos necessários.

**Você irá aprender a:**
- Construir a arquitetura geral de um algoritmo que aprende por experiência (regressão logística), incluindo:
    - Inicialização dos parâmetros
    - Utilização de um algoritmo de otimização

## 1 - Conexão ao Google Drive e Pacotes ##

Primeiramente, vamos executar as células abaixo para conectar ao Google Drive, importar todos os pacotes que você precisará durante o exercício, e "compilar" a função que carrega a base de dados (treino e teste).
- [numpy](www.numpy.org) é um pacote fundamental para computação científica em Python.
- [h5py](http://www.h5py.org) é um pacote comum para interagir com uma base de dados armazenada em um arquivo H5.
- [matplotlib](http://matplotlib.org) é uma biblioteca famosa para plotar gráficos em Python.
- [PIL](http://www.pythonware.com/products/pil/) e [scipy](https://www.scipy.org/) são usados aqui para testar o seu modelo vetorizado com sua própria imagem ao fim do exercício.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# 1 - Dê a permissão
# 2 - Copie e cole o código gerado
# 3 - Coloque as pastas "datasets" e "images" no diretório raíz do drive

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import PIL
import scipy
from scipy import ndimage

%matplotlib inline

In [ ]:
def load_dataset():
    train_dataset = h5py.File('/content/drive/My Drive/datasets/train_catvnoncat.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('/content/drive/My Drive/datasets/test_catvnoncat.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes

    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))

    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

## 2 - Visão geral do problema ##

**Enunciado do Problema**: Lhe foi dada uma base de dados ("data.h5") contendo:

- Um conjunto de treinamento com m_train imagens rotuladas como cat (y=1) ou non-cat (y=0)

- Um conjunto de teste com m_test imagens rotuladas como cat ou non-cat

- Cada imagem possui as dimensões (num_px, num_px, 3), onde 3 é para os 3 canais RGB. Assim, cada imagem é quadrada (altura = num_px e largura = num_px).

Você irá construir um simples algoritmo reconhecedor de imagens que pode classificar corretamente imagens como cat ou non-cat.

Vamos ganhar familiaridade com a base de dados. Carregue os dados executando a célula a seguir.

In [ ]:
# Carrega os dados (cat/non-cat)
train_set_x_orig, train_set_y, test_set_x_orig, test_set_y, classes = load_dataset()

Nós adicionamos "_orig" no final da base de dados de imagens (train e test) porque iremos realizar um pré-processamento nelas. Depois do pré-processamento, ficaremos com train_set_x e test_set_x (os rótulos train_set_y e test_set_y não necessitam de qualquer pré-processamento).

Cada linha de train_set_x_orig e test_set_x_orig é um vetor representando uma imagem. Você pode visualizar um exemplo executando a célula abaixo. Sinta-se a vontade também para modificar o valor da variável `index` e executar novamente para ver outra imagem.

In [ ]:
# Exemplo de imagem
index = 7
plt.imshow(train_set_x_orig[index])
print ("y = " + str(train_set_y[:, index]) + ", é uma imagem '" + classes[np.squeeze(train_set_y[:, index])].decode("utf-8") +  "'.")

Vários bugs em vetorização advêm de problemas com dimensões de matrizes e vetores que não combinam. Se você puder se certificar que as dimensões de suas matrizes/vetores estão corretas, você eliminará vários bugs.

**Exercício:** Encontre os valores para:
  - m_train (número de exemplos de treinamento)
  - m_test (número de exemplos de teste)
  - num_px (= altura = largura da imagem de treinamento)

Lembre-se que `train_set_x_orig` é um vetor numpy com shape (m_train, num_px, num_px, 3). Por exemplo, você pode acessar `m_train` escrevendo `train_set_x_orig.shape[0]`.

In [ ]:
# m_train =
# m_test =
# num_px =

print ("Número de exemplos para treino: m_train = " + str(m_train))
print ("Número de exemplos para teste: m_test = " + str(m_test))
print ("Altura/Largura de cada imagem: num_px = " + str(num_px))
print ("Cada imagem tem o tamanho de: (" + str(num_px) + ", " + str(num_px) + ", 3)")
print ("train_set_x shape: " + str(train_set_x_orig.shape))
print ("train_set_y shape: " + str(train_set_y.shape))
print ("test_set_x shape: " + str(test_set_x_orig.shape))
print ("test_set_y shape: " + str(test_set_y.shape))

**Saída esperada para m_train, m_test and num_px**:
<table style="width:15%">
  <tr>
    <td>**m_train**</td>
    <td> 209 </td>
  </tr>
  
  <tr>
    <td>**m_test**</td>
    <td> 50 </td>
  </tr>
  
  <tr>
    <td>**num_px**</td>
    <td> 64 </td>
  </tr>
  
</table>


Por conveniência, você deve agora reorganizar as dimensões originais (num_px, num_px, 3) em um vetor numpy de dimensões (num_px $*$ num_px $*$ 3, 1). Depois disso, nossa base de dados de treinamento (e de teste) será composta por vetores-coluna, onde cada coluna representa uma imagem. Devem haver m_train (respectivamente m_test) colunas.

**Exercício:** Reorganize as bases de dados de treinamento e teste tal que imagens de dimensões (num_px, num_px, 3) são reorganizadas em vetores-coluna com dimensões (num\_px $*$ num\_px $*$ 3, 1).

In [ ]:
# Reshape dos exemplos de treino e teste

train_set_x_flatten = train_set_x_orig.reshape(train_set_x_orig.shape[0], -1).T
test_set_x_flatten = test_set_x_orig.reshape(test_set_x_orig.shape[0], -1).T

print ("train_set_x_flatten shape: " + str(train_set_x_flatten.shape))
print ("train_set_y shape: " + str(train_set_y.shape))
print ("test_set_x_flatten shape: " + str(test_set_x_flatten.shape))
print ("test_set_y shape: " + str(test_set_y.shape))
print ("sanity check after reshaping: " + str(train_set_x_flatten[0:5,0]))

**Expected Output**:

<table style="width:35%">
  <tr>
    <td>**train_set_x_flatten shape**</td>
    <td> (12288, 209)</td>
  </tr>
  <tr>
    <td>**train_set_y shape**</td>
    <td>(1, 209)</td>
  </tr>
  <tr>
    <td>**test_set_x_flatten shape**</td>
    <td>(12288, 50)</td>
  </tr>
  <tr>
    <td>**test_set_y shape**</td>
    <td>(1, 50)</td>
  </tr>
  <tr>
  <td>**sanity check after reshaping**</td>
  <td>[17 31 56 22 33]</td>
  </tr>
</table>

Para representar imagens coloridas, os canais vermelho, verde e azul (RGB) devem ser especificados para cada pixel, e então o valor do pixel é na verdade um vetor de três números variando de 0 até 255.

Uma etapa de pré-processamento comum é centralizar e padronizar a base de dados, o que significa que você subtrairá a média (de todo o vetor numpy) de cada exemplo, e então dividirá cada exemplo pelo desvio padrão (de todo o vetor numpy). Porém, para bases de dados de imagens, é mais simples e conveniente dividir todas as linhas da base de dados por 255 (o valor máximo de um canal de pixels).

<!-- Durante o treinamento do seu algoritmo, você irá multiplicar pesos e adicionar bias a algumas entradas iniciais para observar ativações de neurônios. Então você retropropagará (backpropagate) com os gradientes para treinar o algoritmo. Contudo, é extremamente importante para cada característica estar em um intervalo similar tal que os gradientes não "explodam". !-->

Vamos padronizar nossa base de dados.

In [ ]:
train_set_x = train_set_x_flatten/255.
test_set_x = test_set_x_flatten/255.

<font color='blue'>
**O que você precisa relembrar:**

Passos comuns para pré-processar uma nova base de dados são:
- Verificar as dimensões (m_train, m_test, num_px, ...)
- Organizar as bases de dados de treino e teste tal que cada exemplo é agora um vetor de dimensões (num_px \* num_px \* 3, 1)
- "Padronizar" seu dados

## 3 - Arquitetura geral do algoritmo ##

Vamos projetar um algoritmo simples para distinguir entre imagens de gatos e outras imagens (não-gato).

Você irá construir um regressão logística, usando a ideia de rede neural. A figura a seguir explicar a razão de **Regressão logística na verdade ser um rede neural simples!**

![alt text](https://static.wixstatic.com/media/84399a_cd34c66a1b9d434ea37417fda650abb3~mv2.png/v1/fill/w_750,h_583,al_c,q_85,usm_0.66_1.00_0.01/84399a_cd34c66a1b9d434ea37417fda650abb3~mv2.webp)


In [ ]:
from sklearn.linear_model import LogisticRegression

# Organiza os dados no formato do framework sklearn
X_treino = train_set_x.T
y_treino = np.squeeze(train_set_y)
X_teste = test_set_x.T
y_teste = np.squeeze(test_set_y)

# Cria um modelo de regressao logistica e o treina com os dados (X, y)
logreg = LogisticRegression()
logreg.fit(X_treino, y_treino)

In [ ]:
#TREINO
from sklearn.metrics import classification_report

ytr = y_treino
ytr_pred = logreg.predict(X_treino)
print(classification_report(ytr, ytr_pred, target_names=classes))

In [ ]:
#TESTE
from sklearn.metrics import classification_report

yte = y_teste
yte_pred = logreg.predict(X_teste)
print(classification_report(yte, yte_pred, target_names=classes))

In [ ]:
#visualizacao da matriz de confusao
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

cm = confusion_matrix(yte, yte_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=logreg.classes_)
disp.plot()

## 4 - Teste com sua própria imagem ##

Você pode usar sua própria imagem e verificar a saída do seu modelo. Para fazer isso:
    
  1. Altere a variável `my_image` para o nome de uma imagem que esteja dentro da pasta `images` (pasta que está no seu drive).
  2. Execute o código e verifique se o algoritmo acerta (1 = cat, 0 = non-cat).

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

## (PUT YOUR IMAGE NAME)
#my_image = "my_image.jpg"   # change this to the name of your image file
my_image = "my_image2.jpg"   # change this to the name of your image file
#my_image = "la_defense.jpg"   # change this to the name of your image file
#my_image = "gargouille.jpg"   # change this to the name of your image file
#my_image = "cat_in_iran.jpg"   # change this to the name of your image file

# Processa nova imagem para ser entrada do algoritmo
fname = "/content/drive/My Drive/images/" + my_image
img = load_img(fname, target_size = (num_px, num_px))
my_image = img_to_array(img).reshape((1, num_px*num_px*3)).T
X_nova_img = my_image.T
my_predicted_image = logreg.predict(X_nova_img) #predict diferente de score()

plt.imshow(img)
print("y = " + str(np.squeeze(my_predicted_image)) + ", seu algoritmo preveu uma imagem \"" + classes[int(np.squeeze(my_predicted_image)),].decode("utf-8") +  "\".")